### Data Pre-Processing

In [5]:
import pandas as pd

def load_bio_file(file_path):
    sentences = []
    sentence = []

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line == '':
                if sentence:
                    sentences.append(sentence)
                    sentence = []
                continue
            parts = line.split()
            if len(parts) == 2:
                token, tag = parts
                sentence.append((token, tag))

    if sentence:
        sentences.append(sentence)

    return sentences

def convert_to_dataframe(sentences):
    data = {
        "sentence_id": [],
        "word": [],
        "tag": []
    }
    for i, sentence in enumerate(sentences):
        for token, tag in sentence:
            data["sentence_id"].append(i)
            data["word"].append(token)
            data["tag"].append(tag)
    return pd.DataFrame(data)

# 📌 Replace this with your actual file path
file_path = "APTNERtrain.txt"

# 🔧 Load and convert the BIO data
sentences = load_bio_file(file_path)
df = convert_to_dataframe(sentences)

# ✅ Save to CSV for future use
df.to_csv("ner_data_train.csv", index=False)

# ✅ Show entity tag types
print("Unique Tags:", sorted(df['tag'].unique()))
print("Sample:")
print(df.head())

Unique Tags: ['B-ACT', 'B-APT', 'B-EMAIL', 'B-ENCR', 'B-FILE', 'B-IDTY', 'B-IP', 'B-LOC', 'B-MAL', 'B-OS', 'B-PROT', 'B-SECTEAM', 'B-SHA2', 'B-TIME', 'B-TOOL', 'B-URL', 'B-VULID', 'B-VULNAME', 'E-ACT', 'E-APT', 'E-EMAIL', 'E-ENCR', 'E-FILE', 'E-IDTY', 'E-IP', 'E-LOC', 'E-MAL', 'E-OS', 'E-PROT', 'E-S-SECTEAM', 'E-SECTEAM', 'E-SHA2', 'E-TIME', 'E-TOOL', 'E-URL', 'E-VULNAME', 'I-ACT', 'I-APT', 'I-FILE', 'I-IDTY', 'I-LOC', 'I-MAL', 'I-OS', 'I-PROT', 'I-SECTEAM', 'I-TIME', 'I-TOOL', 'I-URL', 'I-VULNAME', 'O', 'PROT', 'S-ACT', 'S-APT', 'S-DOM', 'S-EMAIL', 'S-ENCR', 'S-FILE', 'S-IDTY', 'S-IP', 'S-LOC', 'S-MAL', 'S-MD5', 'S-OS', 'S-PROT', 'S-S-SECTEAM', 'S-SECTEAM', 'S-SHA1', 'S-SHA2', 'S-TIME', 'S-TOOL', 'S-URL', 'S-VULID', 'S-VULNAME']
Sample:
   sentence_id   word     tag
0            0   From       O
1            0  April  B-TIME
2            0  19-24  I-TIME
3            0      ,  I-TIME
4            0   2017  E-TIME


In [6]:
file_path = "APTNERtest.txt"

# 🔧 Load and convert the BIO data
sentences = load_bio_file(file_path)
df = convert_to_dataframe(sentences)

# ✅ Save to CSV for future use
df.to_csv("ner_data_test.csv", index=False)

# ✅ Show entity tag types
print("Unique Tags:", sorted(df['tag'].unique()))
print("Sample:")
print(df.head())

Unique Tags: ['B-ACT', 'B-APT', 'B-FILE', 'B-IDTY', 'B-LOC', 'B-MAL', 'B-OS', 'B-PROT', 'B-SECTEAM', 'B-TIME', 'B-TOOL', 'B-VULID', 'B-VULNAME', 'E-ACT', 'E-APT', 'E-FILE', 'E-IDTY', 'E-LOC', 'E-MAL', 'E-OS', 'E-PROT', 'E-SECTEAM', 'E-TIME', 'E-TOOL', 'E-VULID', 'E-VULNAME', 'I-ACT', 'I-APT', 'I-FILE', 'I-IDTY', 'I-LOC', 'I-MAL', 'I-OS', 'I-PROT', 'I-SECTEAM', 'I-TIME', 'I-TOOL', 'O', 'S-ACT', 'S-APT', 'S-DOM', 'S-EMAIL', 'S-ENCR', 'S-FILE', 'S-IDTY', 'S-IP', 'S-LOC', 'S-MAL', 'S-MD5', 'S-OS', 'S-PROT', 'S-SECTEAM', 'S-SHA2', 'S-TIME', 'S-TOOL', 'S-URL', 'S-VULID', 'S-VULNAME']
Sample:
   sentence_id         word tag
0            0          One   O
1            0  certificate   O
2            0          was   O
3            0    generated   O
4            0      locally   O


### MEMM

In [7]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn_crfsuite import CRF, metrics as crf_metrics
import nltk
from nltk.tag import hmm
import warnings
warnings.filterwarnings("ignore")

In [18]:
train_df = pd.read_csv("ner_data_train.csv")
test_df = pd.read_csv("ner_data_test.csv")

In [19]:
train_df.dropna(subset=["word", "tag"], inplace=True)
test_df.dropna(subset=["word", "tag"], inplace=True)

In [20]:
def word2features(sent, i):
    word = str(sent[i][0]) if sent[i][0] is not None else ""

    features = {
        'word': word,
        'is_upper': word.isupper(),
        'is_title': word.istitle(),
        'is_digit': word.isdigit()
    }

    # Previous word
    if i > 0:
        prev_word = str(sent[i - 1][0]) if sent[i - 1][0] is not None else ""
        features.update({
            '-1:word': prev_word,
            '-1:is_title': prev_word.istitle()
        })
    else:
        features['BOS'] = True

    # Next word
    if i < len(sent) - 1:
        next_word = str(sent[i + 1][0]) if sent[i + 1][0] is not None else ""
        features.update({
            '+1:word': next_word,
            '+1:is_title': next_word.istitle()
        })
    else:
        features['EOS'] = True

    return features

def prepare_data(df):
    grouped = df.groupby("sentence_id").apply(lambda x: list(zip(x["word"], x["tag"])))
    return list(grouped)

def extract_features_labels(sentences):
    X, y = [], []
    for sent in sentences:
        X.append([word2features(sent, i) for i in range(len(sent))])
        y.append([tag for _, tag in sent])
    return X, y

In [21]:
train_sents = prepare_data(train_df)
test_sents = prepare_data(test_df)
X_train_feats, y_train = extract_features_labels(train_sents)
X_test_feats, y_test = extract_features_labels(test_sents)

In [22]:
X_flat = [f for seq in X_train_feats for f in seq]
y_flat = [t for seq in y_train for t in seq]
X_test_flat = [f for seq in X_test_feats for f in seq]
y_test_flat = [t for seq in y_test for t in seq]

vec = DictVectorizer()
X_train_vec = vec.fit_transform(X_flat)
X_test_vec = vec.transform(X_test_flat)

all_labels = y_flat + y_test_flat
le = LabelEncoder()
le.fit(all_labels)

y_train_enc = le.transform(y_flat)
y_test_enc = le.transform(y_test_flat)

clf = LogisticRegression(max_iter=200)
clf.fit(X_train_vec, y_train_enc)
y_pred_enc = clf.predict(X_test_vec)
y_pred = le.inverse_transform(y_pred_enc)

print("📌 MEMM Results:\n")
print(classification_report(y_test_flat, y_pred))

📌 MEMM Results:

              precision    recall  f1-score   support

       B-ACT       0.10      0.80      0.18        10
       B-APT       0.15      0.67      0.24        12
      B-FILE       0.00      0.00      0.00         6
      B-IDTY       0.03      0.08      0.04        25
       B-LOC       0.54      0.73      0.62        26
       B-MAL       0.06      0.27      0.10        15
        B-OS       0.00      0.00      0.00         4
      B-PROT       0.00      0.00      0.00         2
   B-SECTEAM       0.72      0.54      0.62        24
      B-TIME       0.82      0.49      0.61       177
      B-TOOL       0.50      0.09      0.15       135
     B-VULID       0.00      0.00      0.00         3
   B-VULNAME       0.00      0.00      0.00         1
       E-ACT       0.05      0.25      0.08        12
       E-APT       0.15      0.67      0.24        12
      E-FILE       0.00      0.00      0.00         6
      E-IDTY       0.04      0.12      0.05        25
       E-L

### CRF Model

In [23]:
crf = CRF(algorithm='lbfgs', max_iterations=100, all_possible_transitions=True)
crf.fit(X_train_feats, y_train)
y_pred_crf = crf.predict(X_test_feats)

print("📌 CRF Results:\n")
print(crf_metrics.flat_classification_report(y_test, y_pred_crf))

📌 CRF Results:

              precision    recall  f1-score   support

       B-ACT       0.12      0.80      0.21        10
       B-APT       0.15      0.83      0.26        12
      B-FILE       0.00      0.00      0.00         6
      B-IDTY       0.03      0.12      0.05        25
       B-LOC       0.55      0.69      0.61        26
       B-MAL       0.04      0.20      0.07        15
        B-OS       1.00      0.25      0.40         4
      B-PROT       0.00      0.00      0.00         2
   B-SECTEAM       0.74      0.58      0.65        24
      B-TIME       0.80      0.58      0.68       177
      B-TOOL       0.48      0.12      0.19       135
     B-VULID       0.00      0.00      0.00         3
   B-VULNAME       0.00      0.00      0.00         1
       E-ACT       0.09      0.50      0.15        12
       E-APT       0.12      0.67      0.21        12
      E-FILE       0.00      0.00      0.00         6
      E-IDTY       0.03      0.12      0.05        25
       E-LO

In [24]:
train_data_hmm = [[(w, t) for w, t in sent] for sent in train_sents]
test_data_hmm = [[w for w, t in sent] for sent in test_sents]
test_tags_hmm = [[t for w, t in sent] for sent in test_sents]

hmm_trainer = hmm.HiddenMarkovModelTrainer()
hmm_model = hmm_trainer.train_supervised(train_data_hmm)

hmm_pred = [hmm_model.tag(sent) for sent in test_data_hmm]
y_pred_hmm = [[tag for _, tag in sent] for sent in hmm_pred]

y_true = [t for seq in test_tags_hmm for t in seq]
y_pred = [t for seq in y_pred_hmm for t in seq]

print("📌 HMM Results:\n")
print(classification_report(y_true, y_pred))

📌 HMM Results:

              precision    recall  f1-score   support

       B-ACT       0.05      0.50      0.10        10
       B-APT       0.10      0.75      0.18        12
     B-EMAIL       0.00      0.00      0.00         0
      B-FILE       0.00      0.00      0.00         6
      B-IDTY       0.04      0.16      0.06        25
       B-LOC       0.39      0.58      0.47        26
       B-MAL       0.00      0.00      0.00        15
        B-OS       1.00      0.50      0.67         4
      B-PROT       0.00      0.00      0.00         2
   B-SECTEAM       0.63      0.50      0.56        24
      B-TIME       0.78      0.38      0.52       177
      B-TOOL       0.20      0.07      0.11       135
     B-VULID       0.00      0.00      0.00         3
   B-VULNAME       0.00      0.00      0.00         1
       E-ACT       0.03      0.25      0.06        12
       E-APT       0.10      0.75      0.18        12
     E-EMAIL       0.00      0.00      0.00         0
      E-FIL

In [25]:
import joblib

# Save the MEMM model, vectorizer, and label encoder
joblib.dump(clf, "memm_model.joblib")
joblib.dump(vec, "memm_vectorizer.joblib")
joblib.dump(le, "memm_label_encoder.joblib")

['memm_label_encoder.joblib']

In [26]:
import pickle

with open("crf_model.pkl", "wb") as f:
    pickle.dump(crf, f)

In [30]:
import dill

with open("hmm_model.dill", "wb") as f:
    dill.dump(hmm_model, f)

In [32]:
os_tokens = [word for word, tag in X_flat if "OS" in tag]
print(set(os_tokens))

ValueError: too many values to unpack (expected 2)

In [33]:
# If using HMM format like [[(word, tag), ...], ...]
os_tokens = [word for sent in train_data_hmm for word, tag in sent if "OS" in tag]
print("OS-related tokens in training data:", set(os_tokens))

OS-related tokens in training data: {'Win64', 'X', 'Linux-based', 'Creators', 'Xp/2003', 'Unix-based', 'Update', 'OSX', 'OS', 'operating', 'XP', 'Windows-based', 'macOS', '%ALLUSERSPROFILE%\\Windows', 'MS', 'Unix', 'Apple', 'MAC', 'Microsoft\\Windows', 'Mac', 'windows', 'The', 'HKLM\\SOFTWARE\\Microsoft\\Windows', 'SysWoW64', 'systems', 'Android', '’s', 'Win32', 'the', 'UNIX', 'MacOS', 'Unix-', 'Win', 'Linux', '10', 'Windows', 'Linux-'}


In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import time

def fetch_cve_data_with_apikey(api_key):
    base_url = "https://services.nvd.nist.gov/rest/json/cves/2.0"
    results_per_page = 2000
    start_index = 0
    all_cve_items = []

    headers = {
        "apiKey": api_key
    }

    # Initial request to get totalResults
    params = {
        "resultsPerPage": results_per_page,
        "startIndex": start_index
    }

    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"Initial request failed with code {response.status_code}")
    
    total = response.json().get("totalResults", 0)
    print(f"Total CVEs to fetch: {total}")

    with tqdm(total=total, desc="Downloading CVEs") as pbar:
        while start_index < total:
            params["startIndex"] = start_index
            resp = requests.get(base_url, headers=headers, params=params)

            # Handle throttling
            if resp.status_code == 429:
                print("Rate limit hit. Sleeping for 10 seconds...")
                time.sleep(10)
                continue
            elif resp.status_code != 200:
                print(f"[ERROR] Failed at index {start_index}, status {resp.status_code}")
                break

            data = resp.json()
            vulns = data.get("vulnerabilities", [])

            for item in vulns:
                cve = item.get("cve", {})
                cve_id = cve.get("id", "")
                source = cve.get("sourceIdentifier", "")
                published = cve.get("published", "")
                modified = cve.get("lastModified", "")
                status = cve.get("vulnStatus", "")
                
                # Extract only English description
                descs = cve.get("descriptions", [])
                desc_en = next((d["value"] for d in descs if d["lang"] == "en"), "")
                
                # CVSS data
                metrics = cve.get("metrics", {})
                cvss_version, vectorString, baseScore, severity = None, None, None, None

                if "cvssMetricV31" in metrics:
                    cvss = metrics["cvssMetricV31"][0]["cvssData"]
                    cvss_version = "3.1"
                elif "cvssMetricV30" in metrics:
                    cvss = metrics["cvssMetricV30"][0]["cvssData"]
                    cvss_version = "3.0"
                elif "cvssMetricV2" in metrics:
                    cvss = metrics["cvssMetricV2"][0]["cvssData"]
                    cvss_version = "2.0"
                else:
                    cvss = {}

                vectorString = cvss.get("vectorString", None)
                baseScore = cvss.get("baseScore", None)
                severity = cvss.get("baseSeverity", None)

                all_cve_items.append({
                    "cve_id": cve_id,
                    "sourceIdentifier": source,
                    "published": published,
                    "lastModified": modified,
                    "vulnStatus": status,
                    "description_en": desc_en,
                    "cvss_version": cvss_version,
                    "vectorString": vectorString,
                    "baseScore": baseScore,
                    "baseSeverity": severity
                })

            fetched = len(vulns)
            start_index += fetched
            pbar.update(fetched)

            # Optional short sleep (uncomment if needed)
            # time.sleep(0.1)

    return pd.DataFrame(all_cve_items)

# ✅ Fetch and save using your API key
api_key = "6f6060da-7e55-441c-b207-5878960b23e8"
df = fetch_cve_data_with_apikey(api_key)
df.to_csv("nvd_cve_cleaned.csv", index=False)
print("✅ Saved cleaned CVE data to: nvd_cve_cleaned.csv")

Total CVEs to fetch: 291403


✅ Saved cleaned CVE data to: nvd_cve_cleaned.csv


In [2]:
import pandas as pd
import time
import re
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import pipeline

# ------------------------- CONFIGURATION -------------------------
ENTITY_FIELDS = [
    "CVE_ID", "Vendor", "Product", "Version", "Operating_System", "Hardware",
    "Vulnerability_Type", "Attack_Vector", "Privileges_Required",
    "User_Interaction", "Exploit_Availability", "Impact",
    "Confidentiality_Impact", "Integrity_Impact", "Availability_Impact",
    "Authentication"
]
MODEL_NAME = "Jean-Baptiste/roberta-large-ner-english"
CHUNK_SIZE = 500
BATCH_WORKERS = 4
# -----------------------------------------------------------------

# Load the Hugging Face pipeline
print("[INFO] Loading NER model...")
ner_pipeline = pipeline("ner", model=MODEL_NAME, grouped_entities=True)
print("[INFO] Model loaded.")


def process_description(description):
    if not isinstance(description, str) or len(description.strip()) == 0:
        return {}

    try:
        entities = ner_pipeline(description)
        formatted = {field: [] for field in ENTITY_FIELDS}

        for entity in entities:
            word = entity["word"]
            label = entity["entity_group"]

            # Heuristic mapping (customize as needed)
            if "ORG" in label or "MISC" in label:
                formatted["Vendor"].append(word)
            elif "PER" in label:
                formatted["User_Interaction"].append(word)
            elif "LOC" in label:
                formatted["Attack_Vector"].append(word)
            else:
                formatted["Vulnerability_Type"].append(word)

        return {k: ", ".join(v) if v else "null" for k, v in formatted.items()}

    except Exception as e:
        print(f"[ERROR] NER failed: {e}")
        return {"error": str(e)}


def threaded_batch_process(df_chunk, max_workers=BATCH_WORKERS):
    results = [None] * len(df_chunk)

    def task(i, desc):
        return i, process_description(desc)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(task, i, desc) for i, desc in enumerate(df_chunk["description_en"])]
        for future in tqdm(as_completed(futures), total=len(futures), desc="Batch"):
            try:
                i, result = future.result()
                results[i] = result
                time.sleep(0.05)
            except Exception as e:
                print("[ERROR] Thread failed:", e)
                results[i] = {"error": str(e)}
    return results


def expand_results(df):
    for field in ENTITY_FIELDS:
        df[field] = df["cybersec_entities"].apply(
            lambda x: x.get(field, "") if isinstance(x, dict) else "")
    return df


def run_pipeline(input_csv_path, output_csv_path, resume=False):
    df = pd.read_csv(input_csv_path)

    if resume and os.path.exists(output_csv_path):
        print(f"[INFO] Resuming from {output_csv_path}")
        df_out = pd.read_csv(output_csv_path)
    else:
        df_out = df.copy()
        df_out["cybersec_entities"] = [{} for _ in range(len(df))]

    start_idx = len(df_out[df_out["cybersec_entities"].apply(lambda x: isinstance(x, dict) and bool(x))])

    for i in range(len(df) - CHUNK_SIZE, -1, -CHUNK_SIZE):
        if i < start_idx:
            continue

        print(f"[INFO] Processing chunk {i} to {i + CHUNK_SIZE}")
        chunk = df.iloc[i:i + CHUNK_SIZE]
        result_chunk = threaded_batch_process(chunk)

        df_out.loc[i:i + CHUNK_SIZE - 1, "cybersec_entities"] = result_chunk
        df_out = expand_results(df_out)

        df_out.to_csv(output_csv_path, index=False)
        print(f"[INFO] Saved progress up to row {i + CHUNK_SIZE}")

    print(f"[DONE] All records processed and saved to {output_csv_path}")
    return df_out

[INFO] Loading NER model...


config.json:   0%|          | 0.00/849 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use mps:0


[INFO] Model loaded.


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [4]:
output = run_pipeline(
    input_csv_path="nvd_cve_cleaned.csv",
    output_csv_path="output.csv",
    resume=True
)
output.head()

[INFO] Processing chunk 290903 to 291403


Batch: 100%|██████████| 500/500 [00:32<00:00, 15.59it/s]


[INFO] Saved progress up to row 291403
[INFO] Processing chunk 290403 to 290903


Batch: 100%|██████████| 500/500 [00:27<00:00, 18.06it/s]


[INFO] Saved progress up to row 290903
[INFO] Processing chunk 289903 to 290403


...

Batch: 100%|██████████| 500/500 [00:28<00:00, 17.32it/s]


[INFO] Saved progress up to row 1403
[INFO] Processing chunk 403 to 903


Batch: 100%|██████████| 500/500 [00:31<00:00, 15.90it/s]


[INFO] Saved progress up to row 903
[DONE] All records processed and saved to output.csv


,cve_id,sourceIdentifier,published,lastModified,vulnStatus,description_en,cvss_version,vectorString,baseScore,baseSeverity,...,Vulnerability_Type,Attack_Vector,Privileges_Required,User_Interaction,Exploit_Availability,Impact,Confidentiality_Impact,Integrity_Impact,Availability_Impact,Authentication
0,CVE-1999-0095,cve@mitre.org,1988-10-01T04:00:00.000,2025-04-03T01:03:51.193,Deferred,"The debug command in Sendmail is enabled, allo...",2.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,10.0,NaN,...,,,,,,,,,,
1,CVE-1999-0082,cve@mitre.org,1988-11-11T05:00:00.000,2025-04-03T01:03:51.193,Deferred,CWD ~root command in ftpd allows root access.,2.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,10.0,NaN,...,,,,,,,,,,
2,CVE-1999-1471,cve@mitre.org,1989-01-01T05:00:00.000,2025-04-03T01:03:51.193,Deferred,Buffer overflow in passwd in BSD based operati...,2.0,AV:L/AC:L/Au:N/C:C/I:C/A:C,7.2,NaN,...,,,,,,,,,,
3,CVE-1999-1122,cve@mitre.org,1989-07-26T04:00:00.000,2025-04-03T01:03:51.193,Deferred,Vulnerability in restore in SunOS 4.0.3 and ea...,2.0,AV:L/AC:L/Au:N/C:P/I:P/A:P,4.6,NaN,...,,,,,,,,,,
4,CVE-1999-1467,cve@mitre.org,1989-10-26T04:00:00.000,2025-04-03T01:03:51.193,Deferred,Vulnerability in rcp on SunOS 4.0.x allows rem...,2.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,10.0,NaN,...,,,,,,,,,,


In [15]:
import pandas as pd

df = pd.read_csv("output.csv")

In [6]:
df.tail()

,cve_id,sourceIdentifier,published,lastModified,vulnStatus,description_en,cvss_version,vectorString,baseScore,baseSeverity,...,Vulnerability_Type,Attack_Vector,Privileges_Required,User_Interaction,Exploit_Availability,Impact,Confidentiality_Impact,Integrity_Impact,Availability_Impact,Authentication
291398,CVE-2025-2580,security@wordfence.com,2025-04-25T06:15:45.457,2025-04-25T06:15:45.457,Received,The Contact Form by Bit Form plugin for WordPr...,3.1,CVSS:3.1/AV:N/AC:H/PR:L/UI:N/S:C/C:L/I:L/A:N,4.9,MEDIUM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291399,CVE-2025-3511,Mitsubishielectric.Psirt@yd.MitsubishiElectric...,2025-04-25T06:15:45.753,2025-04-25T06:15:45.753,Received,Improper Validation of Specified Quantity in I...,3.1,CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:N/I:N/A:H,5.9,MEDIUM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291400,CVE-2025-3861,security@wordfence.com,2025-04-25T06:15:45.957,2025-04-25T06:15:45.957,Received,The Prevent Direct Access – Protect WordPress ...,3.1,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:L/I:L/A:N,5.4,MEDIUM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291401,CVE-2025-3923,security@wordfence.com,2025-04-25T06:15:46.117,2025-04-25T06:15:46.117,Received,The Prevent Direct Access – Protect WordPress ...,3.1,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:L/I:N/A:N,5.3,MEDIUM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291402,CVE-2025-46613,cve@mitre.org,2025-04-25T06:15:46.273,2025-04-25T06:15:46.273,Received,OpenPLC 3 through 64f9c11 has server.cpp Memor...,3.1,CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:C/C:N/I:L/A:H,7.5,HIGH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(df.tail())

                cve_id                                   sourceIdentifier  \
291398   CVE-2025-2580                             security@wordfence.com   
291399   CVE-2025-3511  Mitsubishielectric.Psirt@yd.MitsubishiElectric...   
291400   CVE-2025-3861                             security@wordfence.com   
291401   CVE-2025-3923                             security@wordfence.com   
291402  CVE-2025-46613                                      cve@mitre.org   

                      published             lastModified vulnStatus  \
291398  2025-04-25T06:15:45.457  2025-04-25T06:15:45.457   Received   
291399  2025-04-25T06:15:45.753  2025-04-25T06:15:45.753   Received   
291400  2025-04-25T06:15:45.957  2025-04-25T06:15:45.957   Received   
291401  2025-04-25T06:15:46.117  2025-04-25T06:15:46.117   Received   
291402  2025-04-25T06:15:46.273  2025-04-25T06:15:46.273   Received   

                                           description_en  cvss_version  \
291398  The Contact Form by Bit For

In [18]:
df.dropna(subset=["description_en"], inplace=True)  

In [19]:
df.shape

(291403, 27)

In [17]:
df.head()

,cve_id,sourceIdentifier,published,lastModified,vulnStatus,description_en,cvss_version,vectorString,baseScore,baseSeverity,...,Vulnerability_Type,Attack_Vector,Privileges_Required,User_Interaction,Exploit_Availability,Impact,Confidentiality_Impact,Integrity_Impact,Availability_Impact,Authentication
0,CVE-1999-0095,cve@mitre.org,1988-10-01T04:00:00.000,2025-04-03T01:03:51.193,Deferred,"The debug command in Sendmail is enabled, allo...",2.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CVE-1999-0082,cve@mitre.org,1988-11-11T05:00:00.000,2025-04-03T01:03:51.193,Deferred,CWD ~root command in ftpd allows root access.,2.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CVE-1999-1471,cve@mitre.org,1989-01-01T05:00:00.000,2025-04-03T01:03:51.193,Deferred,Buffer overflow in passwd in BSD based operati...,2.0,AV:L/AC:L/Au:N/C:C/I:C/A:C,7.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CVE-1999-1122,cve@mitre.org,1989-07-26T04:00:00.000,2025-04-03T01:03:51.193,Deferred,Vulnerability in restore in SunOS 4.0.3 and ea...,2.0,AV:L/AC:L/Au:N/C:P/I:P/A:P,4.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CVE-1999-1467,cve@mitre.org,1989-10-26T04:00:00.000,2025-04-03T01:03:51.193,Deferred,Vulnerability in rcp on SunOS 4.0.x allows rem...,2.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
